In [8]:
import os
import sqlite3
import pandas as pd
import glob
import zipfile
from datetime import datetime

def get_dbeaver_db_path():
    """Ask user for the path to their DBeaver SQLite database"""
    print("Please provide the path to your DBeaver SQLite database file.")
    print("You can find this by right-clicking on your database in DBeaver,")
    print("selecting 'Edit Connection', and checking the 'Path' field.")
    
    db_path = input("Enter the path to your DBeaver SQLite database: ")
    
    # Validate the path
    if not os.path.exists(db_path):
        print(f"Error: File not found at {db_path}")
        return None
    
    # Verify it's a SQLite database
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("PRAGMA database_list")
        conn.close()
        return db_path
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

def extract_zip_if_needed(zip_file_path, extract_to=None):
    """Extract ZIP file if data folder doesn't exist"""
    if extract_to is None:
        extract_to = os.path.dirname(zip_file_path)
    
    data_folder = os.path.join(extract_to, 'data')
    
    # Check if data folder already exists
    if os.path.exists(data_folder) and os.path.isdir(data_folder):
        csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
        if csv_files:
            print(f"Data folder already exists with {len(csv_files)} CSV files")
            return data_folder
    
    # Extract the ZIP file
    print(f"Extracting {zip_file_path} to {extract_to}...")
    
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print("Extraction completed successfully")
        
        # Check if data folder exists after extraction
        if os.path.exists(data_folder) and os.path.isdir(data_folder):
            csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
            print(f"Extracted {len(csv_files)} CSV files to {data_folder}")
            return data_folder
        else:
            # Look for any CSV files in the extracted content
            csv_files = []
            for root, dirs, files in os.walk(extract_to):
                for file in files:
                    if file.endswith('.csv'):
                        csv_files.append(os.path.join(root, file))
            
            if csv_files:
                print(f"Found {len(csv_files)} CSV files in {extract_to}")
                # Return the directory containing the CSV files
                return os.path.dirname(csv_files[0])
            else:
                print("No CSV files found in the extracted content")
                return None
    except Exception as e:
        print(f"Error extracting ZIP file: {e}")
        return None

def import_csv_to_smart_logs(db_path, csv_file, chunk_size=10000):
    """Import CSV data to the smart_logs table in chunks"""
    print(f"Importing {csv_file} to smart_logs table...")
    
    # Connect to SQLite
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if smart_logs table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='smart_logs'")
    if not cursor.fetchone():
        print("Error: smart_logs table doesn't exist in the database!")
        conn.close()
        return False
    
    # Total rows processed
    total_rows = 0
    start_time = datetime.now()
    
    # Process file in chunks to avoid memory issues
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        # Convert data types if needed
        for col in chunk.columns:
            if col in ['date', 'serial_number', 'model']:
                continue
            
            if chunk[col].dtype == 'object':
                try:
                    chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
                except:
                    pass
        
        # Insert data into smart_logs table
        chunk.to_sql('smart_logs', conn, if_exists='append', index=False)
        
        # Update row count
        total_rows += len(chunk)
        print(f"  Processed {total_rows} rows so far...")
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    conn.close()
    
    print(f"Completed importing {csv_file}: {total_rows} rows in {duration:.2f} seconds")
    return True

def main():
    # Get project root directory
    project_dir = os.getcwd()
    print(f"Current working directory: {project_dir}")
    
    # Get DBeaver database path
    db_path = get_dbeaver_db_path()
    if not db_path:
        return
    
    # Look for ZIP file
    zip_file_path = os.path.join(project_dir, 'data_Q1_2018.zip')
    if os.path.exists(zip_file_path):
        # Extract the ZIP file
        data_folder = extract_zip_if_needed(zip_file_path, project_dir)
    else:
        # Look for data folder
        data_folder = os.path.join(project_dir, 'data')
        if not os.path.exists(data_folder):
            print(f"Data folder not found: {data_folder}")
            # Ask user for data folder path
            data_folder = input("Please enter the path to your data folder: ")
            if not os.path.exists(data_folder):
                print(f"Path not found: {data_folder}")
                return
    
    # Find all CSV files in the data folder
    csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
    if not csv_files:
        print(f"No CSV files found in {data_folder}")
        return
    
    print(f"Found {len(csv_files)} CSV files in {data_folder}")
    
    # Import each file to smart_logs table
    success_count = 0
    for i, csv_file in enumerate(csv_files, 1):
        print(f"Processing file {i}/{len(csv_files)}: {os.path.basename(csv_file)}")
        try:
            if import_csv_to_smart_logs(db_path, csv_file):
                success_count += 1
        except Exception as e:
            print(f"Error processing {csv_file}: {e}")
    
    print(f"Import complete: Successfully imported {success_count} out of {len(csv_files)} files")
    
    # Create indexes if they don't exist
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print("Creating indexes for faster querying (if they don't exist)...")
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_serial_number ON smart_logs (serial_number)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_date ON smart_logs (date)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_failure ON smart_logs (failure)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_model ON smart_logs (model)')
        conn.commit()
        conn.close()
        print("Indexes created successfully")
    except Exception as e:
        print(f"Error creating indexes: {e}")
    
    print(f"Database '{db_path}' is now ready for analysis")

if __name__ == "__main__":
    main()

Current working directory: /home/tanushreehr/Anomaly-Detection/Anomaly-Detection
Please provide the path to your DBeaver SQLite database file.
You can find this by right-clicking on your database in DBeaver,
selecting 'Edit Connection', and checking the 'Path' field.
Error: File not found at 
